<a href="https://colab.research.google.com/github/Mario-td/Hand-Gesture-Classification-with-Tensorflow-2.0/blob/master/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data preparation**

In [11]:
!pip install orderedset
import pandas as pd
import numpy as np
from orderedset import OrderedSet

In [12]:
# Allows access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Path to the files
path = '/content/drive/My Drive/MastersThesis/Dataset/'
df_train3D = pd.read_csv(path + '/TrainingSet3D.csv')
df_test3D = pd.read_csv(path + '/TestSet3D.csv')
df_test3D

,Label,Sequence,j0_x,j0_y,j0_z,j1_x,j1_y,j1_z,j2_x,j2_y,j2_z,j3_x,j3_y,j3_z,j4_x,j4_y,j4_z,j5_x,j5_y,j5_z,j6_x,j6_y,j6_z,j7_x,j7_y,j7_z,j8_x,j8_y,j8_z,j9_x,j9_y,j9_z,j10_x,j10_y,j10_z,j11_x,j11_y,j11_z,j12_x,j12_y,j12_z,j13_x,j13_y,j13_z,j14_x,j14_y,j14_z,j15_x,j15_y,j15_z,j16_x,j16_y,j16_z,j17_x,j17_y,j17_z,j18_x,j18_y,j18_z,j19_x,j19_y,j19_z,j20_x,j20_y,j20_z
0,0,0,367.15625,323.156250,771.0,388.343750,284.312500,0.0,395.406250,252.531250,0.0,409.531250,217.218750,771.0,420.125000,196.031250,771.0,353.031250,238.406250,771.0,0.000000,0.000000,0.0,374.218750,174.843750,771.0,367.156250,171.312500,771.0,0.000000,0.000000,0.0,353.031250,178.375000,771.0,342.437500,178.375000,771.0,328.312500,178.375000,771.0,0.000000,0.000000,0.0,349.500000,181.906250,771.0,335.375000,178.375000,771.0,321.25000,178.37500,771.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,331.843750,181.906250,771.0,314.187500,185.437500,771.0
1,0,1,336.78125,349.593750,771.0,289.453125,338.671875,771.0,256.687500,335.031250,771.0,253.046875,294.984375,2570.0,242.125000,273.140625,2570.0,285.812500,265.859375,2313.0,274.890625,244.015625,2057.0,267.609375,233.093750,2056.0,253.046875,236.734375,2056.0,304.015625,254.937500,0.0,300.375000,233.093750,0.0,296.734375,211.250000,0.0,289.453125,193.046875,0.0,322.218750,258.578125,0.0,314.937500,233.093750,0.0,318.578125,214.890625,771.0,314.93750,196.68750,771.0,344.062500,265.859375,514.0,347.703125,247.656250,771.0,351.343750,233.093750,771.0,347.703125,214.890625,771.0
2,0,2,334.25000,350.250000,771.0,289.250000,339.000000,771.0,251.750000,335.250000,771.0,251.750000,297.750000,2570.0,240.500000,279.000000,2570.0,281.750000,267.750000,257.0,270.500000,245.250000,2056.0,259.250000,230.250000,2313.0,251.750000,211.500000,2056.0,300.500000,260.250000,0.0,293.000000,237.750000,0.0,281.750000,219.000000,0.0,278.000000,196.500000,0.0,315.500000,260.250000,0.0,311.750000,237.750000,0.0,308.000000,215.250000,0.0,308.00000,196.50000,771.0,338.000000,267.750000,0.0,341.750000,249.000000,771.0,341.750000,234.000000,771.0,341.750000,219.000000,771.0
3,0,3,359.62500,326.625000,771.0,319.125000,309.750000,771.0,285.375000,276.000000,0.0,278.625000,252.375000,0.0,278.625000,249.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,352.875000,238.875000,771.0,359.625000,181.500000,771.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,363.000000,245.625000,771.0,356.250000,184.875000,771.0,356.250000,198.375000,771.0,0.00000,0.00000,0.0,379.875000,255.750000,771.0,390.000000,232.125000,771.0,390.000000,218.625000,771.0,386.625000,205.125000,771.0
4,0,4,393.46875,320.218750,771.0,414.656250,281.375000,771.0,418.187500,249.593750,771.0,315.781250,239.000000,771.0,315.781250,228.406250,514.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,351.093750,189.562500,771.0,344.031250,175.437500,771.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,347.56250,175.43750,771.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,365.218750,171.906250,771.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11579,4,27,217.15625,319.265625,1286.0,252.281250,292.921875,0.0,283.015625,266.578125,514.0,291.796875,235.843750,514.0,304.968750,213.890625,514.0,247.890625,209.500000,514.0,256.671875,183.156250,0.0,256.671875,152.421875,0.0,252.281250,130.468750,0.0,225.937500,205.109375,0.0,230.328125,169.984375,0.0,221.546875,148.031250,2827.0,212.765625,130.468750,2570.0,203.984375,209.500000,0.0,195.203125,183.156250,0.0,190.812500,161.203125,2056.0,182.03125,139.25000,2056.0,182.031250,227.062500,514.0,164.468750,205.109375,1799.0,160.078125,191.937500,1799.0,151.296875,178.765625,1543.0


In [14]:
# Gets the number of steps per sample
n_steps = max(df_train3D['Sequence']) + 1

# **Find defective data**

In [15]:
# Find the indexes of the frames with all zeros in the coordinate columns
ind = np.flatnonzero(df_train3D[[i for i in list(df_train3D.columns)
                     if i not in ['Label', 'Sequence']]].eq(0).all(axis=1))

In [16]:
# Finds out which sequences contain more than N frames
# with all zeros in the columns
def findDefaultSequence(ind, n_steps, N):
    # Sequence number
    n = 0
    count = 0
    # Samples with more than N rows with all zeros
    s = OrderedSet()

    for i in ind:
        if int(i/n_steps) != n:
            count = 1
            n = int(i/n_steps)
        else:
            count = count + 1
        if count >= N:
            s.add(int(i/n_steps))
    return s

N = 5
len(findDefaultSequence(ind, n_steps, N))

120

In [17]:
# Drops default samples
for i in reversed(findDefaultSequence(ind, n_steps, N)):  
    df_train3D = df_train3D.drop(np.arange(i * n_steps, i * n_steps + n_steps, 1))

# **Data Normalization**

In [18]:
# Number of hand keypoints
n_keypoints = 21
# Max depth
d16bits = 65538
# Image height and width
img_height = 480
img_width = 640
for i in range(0, n_keypoints):
    df_train3D['j%d_x' % i] = df_train3D['j%d_x' % i] / img_width
    df_train3D['j%d_y' % i] = df_train3D['j%d_y' % i] / img_height
    df_train3D['j%d_z' % i] = df_train3D['j%d_z' % i] / d16bits
    df_test3D['j%d_x' % i] = df_test3D['j%d_x' % i] / img_width
    df_test3D['j%d_y' % i] = df_test3D['j%d_y' % i] / img_height
    df_test3D['j%d_z' % i] = df_test3D['j%d_z' % i] / d16bits
df_train3D

,Label,Sequence,j0_x,j0_y,j0_z,j1_x,j1_y,j1_z,j2_x,j2_y,j2_z,j3_x,j3_y,j3_z,j4_x,j4_y,j4_z,j5_x,j5_y,j5_z,j6_x,j6_y,j6_z,j7_x,j7_y,j7_z,j8_x,j8_y,j8_z,j9_x,j9_y,j9_z,j10_x,j10_y,j10_z,j11_x,j11_y,j11_z,j12_x,j12_y,j12_z,j13_x,j13_y,j13_z,j14_x,j14_y,j14_z,j15_x,j15_y,j15_z,j16_x,j16_y,j16_z,j17_x,j17_y,j17_z,j18_x,j18_y,j18_z,j19_x,j19_y,j19_z,j20_x,j20_y,j20_z
0,0,0,0.427588,0.782585,0.000000,0.487573,0.679753,0.000000,0.538989,0.599772,0.003921,0.564697,0.508366,0.003921,0.607544,0.462663,0.007843,0.453296,0.474089,0.003921,0.453296,0.394108,0.000000,0.453296,0.348405,0.003921,0.453296,0.291276,0.015686,0.410449,0.474089,0.003906,0.401880,0.405534,0.000000,0.401880,0.336979,0.000000,0.393311,0.268424,0.000000,0.367603,0.496940,0.000000,0.350464,0.439811,0.000000,0.350464,0.371257,0.000000,0.350464,0.314128,0.000000,0.341895,0.565495,0.000000,0.307617,0.519792,0.000000,0.299048,0.474089,0.000000,0.307617,0.416960,0.000000
1,0,1,0.376929,0.779590,0.000000,0.444702,0.666634,0.003921,0.470117,0.553678,0.003921,0.487061,0.474609,0.003921,0.520947,0.429427,0.007843,0.368457,0.452018,0.003921,0.368457,0.384245,0.003921,0.359985,0.339062,0.003921,0.351514,0.282585,0.000000,0.326099,0.474609,0.000015,0.317627,0.395540,0.000000,0.300684,0.327767,0.003921,0.283740,0.271289,0.023528,0.292212,0.508496,0.003921,0.266797,0.429427,0.000000,0.249854,0.372949,0.000000,0.249854,0.316471,0.000000,0.275269,0.564974,0.000000,0.215967,0.519792,0.000000,0.207495,0.474609,0.000000,0.207495,0.440723,0.000000
2,0,2,0.364795,0.769401,0.000000,0.416064,0.666862,0.003921,0.450244,0.552930,0.003921,0.467334,0.473177,0.003921,0.492969,0.427604,0.007843,0.347705,0.450391,0.003921,0.339160,0.393424,0.003921,0.330615,0.336458,0.003921,0.313525,0.279492,0.000000,0.304980,0.473177,0.003921,0.296436,0.393424,0.003921,0.270801,0.336458,0.003921,0.253711,0.268099,0.023528,0.270801,0.507357,0.003921,0.236621,0.438997,0.000000,0.219531,0.382031,0.000000,0.210986,0.325065,0.000000,0.253711,0.564323,0.000000,0.202441,0.530143,0.000000,0.185352,0.484570,0.000000,0.176807,0.438997,0.000000
3,0,3,0.349219,0.747461,0.000000,0.400049,0.657096,0.003921,0.442407,0.555436,0.003921,0.467822,0.476367,0.003921,0.484766,0.408594,0.003921,0.340747,0.465072,0.003921,0.323804,0.386003,0.000000,0.306860,0.329525,0.000000,0.289917,0.284342,0.000000,0.289917,0.476367,0.003921,0.264502,0.397298,0.003921,0.247559,0.329525,0.003921,0.222144,0.273047,0.023528,0.256030,0.510254,0.003921,0.222144,0.431185,0.000000,0.205200,0.374707,0.000000,0.196729,0.329525,0.000000,0.239087,0.566732,0.000000,0.188257,0.532845,0.000000,0.171313,0.498958,0.000000,0.162842,0.453776,0.000000
4,0,4,0.342187,0.750000,0.000000,0.393750,0.646875,0.003921,0.428125,0.555208,0.003921,0.436719,0.463542,0.003921,0.453906,0.406250,0.007843,0.316406,0.463542,0.003921,0.299219,0.383333,0.003921,0.282031,0.326042,0.000000,0.264844,0.280208,0.000000,0.273438,0.475000,0.003921,0.247656,0.394792,0.003921,0.221875,0.326042,0.003921,0.196094,0.280208,0.000000,0.239063,0.509375,0.003921,0.204687,0.440625,0.003921,0.178906,0.383333,0.000000,0.161719,0.326042,0.000000,0.221875,0.566667,0.000000,0.178906,0.532292,0.000000,0.153125,0.497917,0.000000,0.135937,0.452083,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115387,4,27,0.594629,0.701237,0.023528,0.634863,0.687826,0.023528,0.670068,0.654297,0.023528,0.685156,0.620768,0.000000,0.675098,0.587240,0.000000,0.624805,0.526888,0.023528,0.619775,0.493359,0.023528,0.624805,0.466536,0.027450,0.609717,0.439714,0.027450,0.584570,0.526888,0.023528,0.579541,0.493359,0.027450,0.569482,0.479948,0.027450,0.559424,0.446419,0.023528,0.564453,0.547005,0.023528,0.544336,0.526888,0.027450,0.534277,0.506771,0.023528,0.524219,0.506771,0.023528,0.529248,0.587240,0.023528,0.524219,

# **2D dataset**

In [21]:
df_train2D = df_train3D[:]
df_test2D = df_test3D[:]
for i in range(0, n_keypoints):
  del df_train2D['j%d_z' % i]
  del df_test2D['j%d_z' % i]
df_test2D

,Label,Sequence,j0_x,j0_y,j1_x,j1_y,j2_x,j2_y,j3_x,j3_y,j4_x,j4_y,j5_x,j5_y,j6_x,j6_y,j7_x,j7_y,j8_x,j8_y,j9_x,j9_y,j10_x,j10_y,j11_x,j11_y,j12_x,j12_y,j13_x,j13_y,j14_x,j14_y,j15_x,j15_y,j16_x,j16_y,j17_x,j17_y,j18_x,j18_y,j19_x,j19_y,j20_x,j20_y
0,0,0,0.573682,0.673242,0.606787,0.592318,0.617822,0.526107,0.639893,0.452539,0.656445,0.408398,0.551611,0.496680,0.000000,0.000000,0.584717,0.364258,0.573682,0.356901,0.000000,0.000000,0.551611,0.371615,0.535059,0.371615,0.512988,0.371615,0.000000,0.000000,0.546094,0.378971,0.524023,0.371615,0.501953,0.371615,0.000000,0.000000,0.000000,0.000000,0.518506,0.378971,0.490918,0.386328
1,0,1,0.526221,0.728320,0.452271,0.705566,0.401074,0.697982,0.395386,0.614551,0.378320,0.569043,0.446582,0.553874,0.429517,0.508366,0.418140,0.485612,0.395386,0.493197,0.475024,0.531120,0.469336,0.485612,0.463647,0.440104,0.452271,0.402181,0.503467,0.538704,0.492090,0.485612,0.497778,0.447689,0.492090,0.409766,0.537598,0.553874,0.543286,0.515951,0.548975,0.485612,0.543286,0.447689
2,0,2,0.522266,0.729688,0.451953,0.706250,0.393359,0.698438,0.393359,0.620313,0.375781,0.581250,0.440234,0.557813,0.422656,0.510938,0.405078,0.479687,0.393359,0.440625,0.469531,0.542188,0.457813,0.495312,0.440234,0.456250,0.434375,0.409375,0.492969,0.542188,0.487109,0.495312,0.481250,0.448437,0.481250,0.409375,0.528125,0.557813,0.533984,0.518750,0.533984,0.487500,0.533984,0.456250
3,0,3,0.561914,0.680469,0.498633,0.645312,0.445898,0.575000,0.435352,0.525781,0.435352,0.518750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.551367,0.497656,0.561914,0.378125,0.000000,0.000000,0.000000,0.000000,0.567187,0.511719,0.556641,0.385156,0.556641,0.413281,0.000000,0.000000,0.593555,0.532813,0.609375,0.483594,0.609375,0.455469,0.604102,0.427344
4,0,4,0.614795,0.667122,0.647900,0.586198,0.653418,0.519987,0.493408,0.497917,0.493408,0.475846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.548584,0.394922,0.537549,0.365495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.543066,0.365495,0.000000,0.000000,0.000000,0.000000,0.570654,0.358138,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11579,4,27,0.339307,0.665137,0.394189,0.610254,0.442212,0.555371,0.455933,0.491341,0.476514,0.445605,0.387329,0.436458,0.401050,0.381576,0.401050,0.317546,0.394189,0.271810,0.353027,0.427311,0.359888,0.354134,0.346167,0.308398,0.332446,0.271810,0.318726,0.436458,0.305005,0.381576,0.298145,0.335840,0.284424,0.290104,0.284424,0.473047,0.256982,0.427311,0.250122,0.399870,0.236401,0.372428
11580,4,28,0.335254,0.625618,0.392969,0.599967,0.441064,0.548665,0.441064,0.484538,0.445874,0.426823,0.392969,0.446061,0.388159,0.369108,0.000000,0.000000,0.000000,0.000000,0.359302,0.439648,0.349683,0.369108,0.344873,0.369108,0.000000,0.000000,0.325635,0.452474,0.311206,0.375521,0.316016,0.381934,0.316016,0.388346,0.301587,0.478125,0.287158,0.433236,0.272729,0.426823,0.272729,0.420410
11581,4,29,0.342871,0.635254,0.401318,0.585156,0.447241,0.540625,0.443066,0.473828,0.430542,0.418164,0.000000,0.000000,0.401318,0.434863,0.405493,0.479395,0.405493,0.523926,0.000000,0.000000,0.376270,0.440430,0.384619,0.512793,0.388794,0.540625,0.000000,0.000000,0.363745,0.434863,0.355396,0.512793,0.359570,0.540625,0.305298,0.529492,0.313647,0.490527,0.326172,0.523926,0.338696,0.540625
11582,4,30,0.338916,0.653060,0.000000,0.000000,0.448340,0.550391,0.452393,0.490951,0.436182,0.431510,0.000000,0.000000,0.419971,0.436914,0.428076,0.474740,0.000000,0.000000,0.000000,0.000000,0.403760,0.436914,0.395654,0.501758,0.403760,0.507161,0.000000,0.000000,0.375391,0.436914,0.371338,0.507161,0.359180,0.528776,0.314600,0.528776,0.318652,0.490951,0.330811,0.523372,0.338916,0.539583


In [22]:
# Saves the dataframes
df_train2D.to_csv(path + '/TrainingSet2D.csv', index=False)
df_train3D.to_csv(path + '/TrainingSet3D.csv', index=False)
df_test2D.to_csv(path + '/TestSet2D.csv', index=False)
df_test3D.to_csv(path + '/TestSet3D.csv', index=False)